In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

In [ ]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [ ]:
df['diagnosis'].value_counts()

,count
diagnosis,
B,357
M,212


In [ ]:
df.drop(columns=['Unnamed: 32','id'],inplace=True)

In [ ]:
df.shape

(569, 31)

In [ ]:
X = df.iloc[:, 1:].values

In [ ]:
y = df.iloc[:, 0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
X_train_tensor.shape

torch.Size([426, 30])

In [ ]:
y_test_tensor

tensor([0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1.,
        0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
        0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.])

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class MyCustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [ ]:
train_dataset = MyCustomDataset(X_train_tensor, y_train_tensor)
test_dataset = MyCustomDataset(X_test_tensor, y_test_tensor)

In [ ]:
train_dataset[10]

(tensor([-0.4503,  1.0953, -0.4610, -0.4714, -1.2617, -0.5829, -0.4472, -0.7675,
         -1.1297, -0.3807, -0.5765, -0.2583, -0.6962, -0.4709, -0.6226, -0.1505,
         -0.0545, -0.5177, -0.8389, -0.1568, -0.4571,  0.9375, -0.5256, -0.4765,
         -0.4683,  0.5573,  0.5460,  0.0212, -0.1454,  0.6134]),
 tensor(0.))

In [ ]:
len(train_dataset)

426

In [ ]:
len(test_dataset)

143

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
class MyNeuralNetwork(nn.Module):

  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [ ]:
model = MyNeuralNetwork(X_train_tensor.shape[1])

In [ ]:
learning_rate = 0.1
epochs = 25

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_function = nn.BCELoss()

In [ ]:
for epoch in range(epochs):
  for features, labels in train_loader:

    y_pred = model(features)

    loss = loss_function(y_pred, labels.view(-1,1))

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  print(f"for epoch{epoch + 1}: loss is {loss.item()}")


for epoch1: loss is 0.12907694280147552
for epoch2: loss is 0.12458387762308121
for epoch3: loss is 0.10733415186405182
for epoch4: loss is 0.08924764394760132
for epoch5: loss is 0.13909052312374115
for epoch6: loss is 0.1307603418827057
for epoch7: loss is 0.060580264776945114
for epoch8: loss is 0.039393115788698196
for epoch9: loss is 0.15059401094913483
for epoch10: loss is 0.14467324316501617
for epoch11: loss is 0.020438339561223984
for epoch12: loss is 0.13646768033504486
for epoch13: loss is 0.18052010238170624
for epoch14: loss is 0.11546085774898529
for epoch15: loss is 0.10539792478084564
for epoch16: loss is 0.0312412828207016
for epoch17: loss is 0.18900921940803528
for epoch18: loss is 0.07462194561958313
for epoch19: loss is 0.5534371733665466
for epoch20: loss is 0.03786482289433479
for epoch21: loss is 0.13945162296295166
for epoch22: loss is 0.27853184938430786
for epoch23: loss is 0.018049165606498718
for epoch24: loss is 0.016085902228951454
for epoch25: loss is 0.